In [1]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

# Load Data

In [61]:
data_path = pathlib.Path('../data')

In [73]:
paper_data = pd.concat([
    pd.read_csv(data_path / 'gidden_et_al_2022_ar6_reanalysis_data.csv'),
    pd.read_csv(data_path / 'additional_analysis_data.csv'),
])

In [75]:
pyam_paper_data = pyam.IamDataFrame(paper_data)
pyam_paper_data.load_meta(data_path / 'gidden_et_al_2022_ar6_reanalysis_meta.csv')
pyam_paper_data.meta = pyam_paper_data.meta.drop(columns=['Category']) # this will come back when we load in AR6 data

pyam.core - INFO: Reading meta indicators for 914 out of 1871 scenarios


In [64]:
ar6_data = pd.read_csv(data_path / 'AR6_Scenarios_Database_World_v1.0.csv')

In [99]:
variables = {
    'AR6 climate diagnostics|Infilled|Emissions|CO2': 
    'AR6 climate diagnostics|Infilled|Emissions|CO2', 
    'AR6 climate diagnostics|Native-with-Infilled|Emissions|Kyoto Gases (AR6-GWP100)': 
    'AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases',
}
pyam_ar6 = pyam.IamDataFrame(ar6_data[ar6_data.Variable.isin(list(variables) + ['Emissions|CO2', 'Emissions|Kyoto Gases'])])
pyam_ar6.load_meta(data_path / 'AR6_Scenarios_Database_metadata_indicators_v1.0.xlsx')
pyam_ar6 = (
    pyam_ar6
    .filter(Category=['C1', 'C3'])
    .rename(variable=variables)
)
columns = {
    'Year of netzero CO2 emissions (Harm-Infilled) table': 
    'Interpolated Net-Zero Year for AR6 climate diagnostics|Infilled|Emissions|CO2',
    'Year of netzero GHG emissions (Harm-Infilled) table': 
    'Interpolated Net-Zero Year for AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases',
}
pyam_ar6.meta.rename(columns=columns, inplace=True)
pyam_ar6.meta = pyam_ar6.meta[['Category'] + list(columns.values())]

pyam.core - WARNING: Reading meta indicators, ignoring 433 scenarios from file


### Combine data sets

We expect to have different numbers of scenarios, since `paper_data` reports only those scenarios which can be run with OSCAR.

In [100]:
df = pyam.concat([pyam_paper_data, pyam_ar6])

In [101]:
len(df.filter(variable='Cumulative AR6 climate diagnostics|Infilled|Emissions|CO2 from 2020').filter(Category='C1').index)

97

In [102]:
len(df.filter(variable='Cumulative AR6 Reanalysis|OSCARv3.2|Emissions|CO2 from 2020').filter(Category='C1').index)

70

In [110]:
variable_templates = [
    'AR6 climate diagnostics|Infilled|Emissions|{gas}', # as reported in AR6
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}|Direct Only', # reanalyzed by OSCAR
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}', # reanalyzed by OSCAR including NGHGI adjustment 
]

# Compute Carbon Budgets

In [111]:
def make_carbon_budget(df, variable):
    meta = (
        df
        .meta
        .query("Category in ['C1', 'C3']")
        [f'Interpolated Net-Zero Year for {variable}']
        .dropna()
        .astype(int)
        .reset_index()
    ) 
    data = (
        df
        .filter(region='World', variable=f'Cumulative {variable} from 2020')
        .convert_unit('Mt CO2/yr', to='Gt CO2', factor=1e-3)
    )
    budgets = [(
        row['model'], 
        row['scenario'], 
        data.filter(
            model=row['model'], scenario=row['scenario'], 
            year=row[f'Interpolated Net-Zero Year for {variable}']
        ).timeseries().values[0][0]
    )
        for i, row in meta.iterrows()
    ]
    return pd.DataFrame(budgets, columns=['model', 'scenario', f'Carbon Budget for {variable}']).set_index(['model', 'scenario'])
        

In [112]:
new_meta = pd.concat([
    make_carbon_budget(df, template.format(gas='CO2')) for template in variable_templates
], axis=1)
df.set_meta(new_meta)
new_meta.head()

Carbon Budget for AR6 climate diagnostics|Infilled|Emissions|CO2  \
model       scenario                                                                                  
AIM/CGE 2.0 SSP1-26                                                       945.615367                  
            SSP4-26                                                       982.207779                  
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         679.395766                  
            CD-LINKS_NPi2020_400                                          508.452135                  
AIM/CGE 2.2 EN_INDCi2030_1000f                                           1160.102947                  

                                   Carbon Budget for AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  \
model       scenario                                                                                      
AIM/CGE 2.0 SSP1-26                                                       862.102117                      
            SSP4-26                                                      1036.223839                      
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         493.573297                      
            CD-LINKS_NPi2020_400                                          364.560104                      
AIM/CGE 2.2 EN_INDCi2030_1000f                                            989.933137                      

                                   Carbon Budget for AR6 Reanalysis|OSCARv3.2|Emissions|CO2  
model       scenario                                                                         
AIM/CGE 2.0 SSP1-26                                                       711.308853         
            SSP4-26                                                       876.356706         
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         374.487241         
            CD-LINKS_NPi2020_400                                          298.821486         
AIM/CGE 2.2 EN_INDCi2030_1000f                                            820.869212

# Emission Reductions

In [113]:
def compute_reduction(df, variable, fromyear=2020, toyear=2030):
    data = (
        df
        .filter(variable=variable, year=[fromyear, toyear], region='World')
        .timeseries()
        .droplevel(['region', 'unit', 'variable'])
    )
    s = (data[fromyear] - data[toyear]) / data[fromyear] * 100
    s.name = f'2030 Emission Reductions for {variable}'
    return s

In [114]:
gases = ['CO2', 'Kyoto Gases']
new_meta = pd.concat([
    compute_reduction(df, template.format(gas=gas))
    for gas, template in itertools.product(gases, variable_templates)
], axis=1)
df.set_meta(new_meta)
new_meta.head()

2030 Emission Reductions for AR6 climate diagnostics|Infilled|Emissions|CO2  \
model       scenario                                                                                             
AIM/CGE 2.0 SSP1-26                                                        20.698212                             
            SSP4-26                                                        23.080940                             
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          39.680445                             
            CD-LINKS_NPi2020_400                                           44.295652                             
            CO_Bridge                                                      22.553044                             

                                   2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  \
model       scenario                                                                                                 
AIM/CGE 2.0 SSP1-26                                                        20.690203                                 
            SSP4-26                                                        19.278438                                 
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          50.210320                                 
            CD-LINKS_NPi2020_400                                           56.814615                                 
            CO_Bridge                                                      28.227574                                 

                                   2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|CO2  \
model       scenario                                                                                     
AIM/CGE 2.0 SSP1-26                                                        24.188745                     
            SSP4-26                                                        22.529429                     
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          56.662343                     
            CD-LINKS_NPi2020_400                                           63.605357                     
            CO_Bridge                                                      33.335398                     

                                   2030 Emission Reductions for AR6 climate diagnostics|Infilled|Emissions|Kyoto Gases  \
model       scenario                                                                                                     
AIM/CGE 2.0 SSP1-26                                                        24.976649                                     
            SSP4-26                                                        28.116878                                     
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          41.311850                                     
            CD-LINKS_NPi2020_400                                           45.320361                                     
            CO_Bridge                                                      27.958333                                     

                                   2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only  \
model       scenario                                                                                                         
AIM/CGE 2.0 SSP1-26                                                        25.684139                                         
            SSP4-26                                                        26.476508                                         
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                          49.059937                                         
            CD-LINKS_NPi2020_400                                           54.422898                                         
            CO_Bridge                                  

In [143]:
def summarize(data, variable_start, gas, name):
    print(name)
    cols = [variable_start + template.format(gas=gas) for template in variable_templates]
    headers = ['AR6 Statistics', 'Pathways with OSCAR Direct Effects', 'Pathways with OSCAR Direct & Indirect Effects']
    for col in cols:
        if not col in data:
            raise ValueError(f'{col} not in data columns')
    return (
        data
        .query("Category in ['C1', 'C3']")
        .replace({'Category': {'C1': '1.5C', 'C3': '2C'}})
        .rename(columns={c: h for c, h in zip(cols, headers)})
        [headers + ['Category']]
        .groupby('Category')
        .describe(percentiles=(0.05, 0.5, 0.95))
        .stack(level=-2)
        [['50%', '5%', '95%']]
        .fillna(2100)
        .astype(int)
        .apply(lambda col: f"{col['50%']} ({col['5%']}-{col['95%']})", axis=1)
        .to_frame(name=name)
        .loc[['1.5C', '2C'], headers, :]
    )

In [144]:
to_summarize = {
    ('Carbon Budget for ', 'CO2'): 'Carbon Budget from 2020',
    ('2030 Emission Reductions for ', 'CO2'): 'CO2 Emissions Reductions (2020-2030)',
    ('Interpolated Net-Zero Year for ', 'CO2'): 'Net-zero CO2 Year',
    ('Interpolated Net-Zero Year for ', 'Kyoto Gases'): 'Net-zero GHG Year',
}

In [145]:
table = pd.concat([
    summarize(df.meta, variable_start, gas, name)
    for (variable_start, gas), name in to_summarize.items()
], axis=1).T
table.to_excel('mitigation_outcomes.xlsx')
table

Carbon Budget from 2020
CO2 Emissions Reductions (2020-2030)
Net-zero CO2 Year
Net-zero GHG Year


C:\Users\gidden\Miniconda3\envs\pyam\lib\site-packages\numpy\lib\function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\gidden\Miniconda3\envs\pyam\lib\site-packages\numpy\lib\function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


Category                                          1.5C  \
                                        AR6 Statistics   
Carbon Budget from 2020                  512 (328-708)   
CO2 Emissions Reductions (2020-2030)        47 (36-69)   
Net-zero CO2 Year                     2052 (2037-2067)   
Net-zero GHG Year                     2098 (2054-2100)   

Category                                                                 \
                                     Pathways with OSCAR Direct Effects   
Carbon Budget from 2020                                   473 (319-620)   
CO2 Emissions Reductions (2020-2030)                         52 (35-67)   
Net-zero CO2 Year                                      2050 (2040-2060)   
Net-zero GHG Year                                      2066 (2051-2087)   

Category                                                                            \
                                     Pathways with OSCAR Direct & Indirect Effects   
Carbon Budget from 2020                                              392 (262-528)   
CO2 Emissions Reductions (2020-2030)                                    56 (39-73)   
Net-zero CO2 Year                                                 2047 (2037-2056)   
Net-zero GHG Year                                                 2067 (2049-2087)   

Category                                            2C  \
                                        AR6 Statistics   
Carbon Budget from 2020                 882 (635-1133)   
CO2 Emissions Reductions (2020-2030)         21 (1-43)   
Net-zero CO2 Year                     2070 (2059-2093)   
Net-zero GHG Year                     2100 (2078-2100)   

Category                                                                 \
                                     Pathways with OSCAR Direct Effects   
Carbon Budget from 2020                                  838 (542-1100)   
CO2 Emissions Reductions (2020-2030)                          21 (1-50)   
Net-zero CO2 Year                                      2068 (2052-2087)   
Net-zero GHG Year                                      2082 (2069-2096)   

Category                                                                            
                                     Pathways with OSCAR Direct & Indirect Effects  
Carbon Budget from 2020                                              703 (445-936)  
CO2 Emissions Reductions (2020-2030)                                     25 (4-55)  
Net-zero CO2 Year                                                 2064 (2049-2083)  
Net-zero GHG Year                                                 2082 (2066-2097)

# Net-zero timings compared to a constant offset of 3.7 Gt CO2

Here we calculate the difference between net-zero timings between the dynamic estimation of indirect effects vs. applying a constant offset value


In [179]:
def calc_nz(df, v):
    data = (
         df
        .filter(region='World', variable=v)
        .filter(year=range(2020, 2101))
        .timeseries()
        .T
        .lt(1e-2)
        .idxmax()
        .reset_index(level=['region', 'variable', 'unit'], drop=True)
    )
    data[data == 2020] = pd.NA
    return data    

In [180]:
v = 'CO2-constant offset'
offsetdf = df.subtract('AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only', 3.7e3, name=v, ignore_units='Mt CO2/yr')
offsetdf.set_meta(calc_nz(offsetdf, v), name=f'Interpolated Net-Zero Year for {v}')

In [181]:
offsetdf.meta['Offset Year Delta'] = (
    offsetdf.meta['Interpolated Net-Zero Year for AR6 Reanalysis|OSCARv3.2|Emissions|CO2'] -
    offsetdf.meta['Interpolated Net-Zero Year for CO2-constant offset']
)

In [182]:
offsetdf.meta.groupby('Category')['Offset Year Delta'].count()

Category
C1     70
C3    194
Name: Offset Year Delta, dtype: int64

In [183]:
(
    offsetdf.meta
    .groupby('Category')['Offset Year Delta']
    .describe(percentiles=(0.05, 0.5, 0.95))
    [['50%', '5%', '95%']]
)

,50%,5%,95%
Category,,,
C1,2.0,0.0,7.0
C3,4.0,1.0,15.0
